# Model Performance

In [ ]:
# load libraries
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import yfinance as yf
from numerapi import NumerAPI
from utils.api_keys import PUBLIC_ID, SECRET_KEY

In [ ]:
# instantiate
TODAY = dt.datetime.now()
nmr = yf.download('NMR-USD', start=TODAY, end=TODAY, progress=False)
napi = NumerAPI(public_id=PUBLIC_ID, secret_key=SECRET_KEY)

In [ ]:
# get model performance history
def get_model_rankings(model_list):
    df_list = []
    for m in model_list:
        df = pd.DataFrame.from_dict(napi.daily_model_performances(m))
        df['modelName'] = m
        df_list.append( df)
    df_all = pd.concat(df_list)
    return df_all

# get model payout history
def get_model_payouts(model_list):
    df_list = []
    for m in model_list:
        df = pd.DataFrame.from_dict(napi.round_model_performances(m))
        df["modelName"] = m
        df_list.append(df)
    df_all = pd.concat(df_list)
    payout_df = pd.pivot_table(
        df_all, 
        values='payout', 
        index=['roundOpenTime'], 
        columns=['modelName']).fillna(0)
    payout_df["totalPayout"] = payout_df.sum(axis=1)
    return payout_df

# plot model rankings
def plot_model_rankings(data, after_date, cols):
    fig, axes = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True, figsize=(12, 8))
    df = data.loc[data["date"] >= after_date].set_index("date")
    df.loc[df["modelName"] == "dh_foxhound", cols].plot(ax=axes[0, 0])
    df.loc[df["modelName"] == "dh_deadcell", cols].plot(ax=axes[0, 1])
    df.loc[df["modelName"] == "dh_cobra", cols].plot(ax=axes[0, 2])
    df.loc[df["modelName"] == "dh_beautybeast", cols].plot(ax=axes[1, 0])
    df.loc[df["modelName"] == "dh_skulls", cols].plot(ax=axes[1, 1])
    df.loc[df["modelName"] == "dh_desperado", cols].plot(ax=axes[1, 2])
    axes[0, 0].set_title("Foxhound")
    axes[0, 1].set_title("Deadcell")
    axes[0, 2].set_title("Cobra")
    axes[1, 0].set_title("BeautyBeast")
    axes[1, 1].set_title("Skulls")
    axes[1, 2].set_title("Desperado")
    plt.show()

---
# Current Staking Amount

In [ ]:
# nmr price
nmr_price = nmr[['Open', 'High', 'Low', 'Close']].mean(axis=1)[0]
nmr_price

In [ ]:
# get model names
model_list = napi.get_models().keys()
model_list

In [ ]:
# get current staking amount
nmr_stake_list = []
nmr_usd_list = []

for k in model_list:
    try:
        nmr_stake = round(float(napi.stake_get(k)), 2)
        nmr_usd = round(nmr_stake * nmr_price, 2)
        nmr_stake_list.append(nmr_stake)
        nmr_usd_list.append(nmr_usd)
    except:
        pass

pd.DataFrame({
    "model": model_list,
    "staked nmr": nmr_stake_list,
    "stked usd": nmr_usd_list
})

---
# Model Ranks

In [ ]:
# get model rankings
perf = get_model_rankings(model_list=model_list)
perf.info()

In [ ]:
# date variable
AFTER_DATE = "2022-04-01"

In [ ]:
# check model name list
perf["modelName"].unique()

In [ ]:
# cols
tc_rank_cols = [col for col in perf.columns if col.endswith("Rank") and col.startswith("tc")]
tc_rep_cols = [col for col in perf.columns if col.endswith("Rep") and col.startswith("tc")]
nontc_rank_cols = [col for col in perf.columns if col.endswith("Rank") and not col.startswith("tc")]
nontc_rep_cols = [col for col in perf.columns if col.endswith("Rep") and not col.startswith("tc")]

tc_rank_cols, tc_rep_cols, nontc_rank_cols, nontc_rep_cols

---
# CorrRank & CorrRep

In [ ]:
# plot  corr rank and rep
corrdf = perf.loc[perf["date"]>=AFTER_DATE]

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=False, figsize=(10, 5))
pd.pivot_table(
    data=corrdf[["modelName", "date", "corrRank"]],
    values="corrRank",
    index="date",
    columns="modelName"
).plot(ax=axes[0])
axes[0].set_title("CORR Ranking (Lower = Better)")

pd.pivot_table(
    data=corrdf[["modelName", "date", "corrRep"]],
    values="corrRep",
    index="date",
    columns="modelName"
).plot(ax=axes[1])
axes[1].set_title("CORR Reputation (Higher = Better)")

plt.tight_layout()
plt.show()

---
# Ranks

In [ ]:
# plot rankings (TC)
plot_model_rankings(data=perf, after_date=AFTER_DATE, cols=tc_rank_cols)

In [ ]:
# plot rankings (non-TC)
plot_model_rankings(data=perf, after_date=AFTER_DATE, cols=nontc_rank_cols)

---
# Reputations

In [ ]:
# plot reputations (TC)
plot_model_rankings(data=perf, after_date=AFTER_DATE, cols=tc_rep_cols)

In [ ]:
# plot reputations (non-TC)
plot_model_rankings(data=perf, after_date=AFTER_DATE, cols=nontc_rep_cols)

---
# Get model payouts

In [ ]:
# get payout
payout = get_model_payouts(model_list)
payout

In [ ]:
# plot payouts
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
payout.loc[AFTER_DATE:][model_list].cumsum().plot(ax=axes[0])
payout.loc[AFTER_DATE:][["totalPayout"]].cumsum().plot(ax=axes[1])
axes[0].set_title("Cumulative Model Payouts")
axes[0].set_xlabel("Round Open Date")
axes[0].set_ylabel("NMR Payout")
axes[1].set_title("Cumulative Total Payouts")
axes[1].set_xlabel("Round Open Date")
axes[1].set_ylabel("NMR Payout")
plt.tight_layout()
plt.show()

---
# Get total return

In [ ]:
# get transactions history
trans = pd.DataFrame(napi.wallet_transactions())
trans["amount"] = trans["amount"].astype("float")
trans.info()

In [ ]:
increase_stake = trans.loc[trans["type"]=="increase_stake"]
increase_stake

In [ ]:
sum_staking = increase_stake["amount"].sum() #- 14 - 14
sum_payout = payout["totalPayout"].sum()
sum_nmr = sum_staking + sum_payout

print(f"Total return to-date: {(sum_payout / sum_nmr * 100).round(1)} %")